In [1]:
import pandas as pd

# Load datasets
old_df = pd.read_csv("/content/old_system_data.csv")
new_df = pd.read_csv("/content/new_crm_data.csv")

# Convert CustomerID to string to avoid issues
old_df['CustomerID'] = old_df['CustomerID'].astype(str)
new_df['CustomerID'] = new_df['CustomerID'].astype(str)

# --- 1. Completeness Check ---
missing_old = old_df.isnull().sum()
missing_new = new_df.isnull().sum()

# --- 2. Accuracy Check (Matching Records) ---
merged = pd.merge(old_df, new_df, on='CustomerID', how='outer', suffixes=('_old', '_new'))

discrepancies = []

for index, row in merged.iterrows():
    for col in ['Name', 'Email', 'Phone']:
        val_old = str(row[col + '_old']) if not pd.isna(row[col + '_old']) else ''
        val_new = str(row[col + '_new']) if not pd.isna(row[col + '_new']) else ''
        if val_old != val_new:
            discrepancies.append({
                'CustomerID': row['CustomerID'],
                'Field': col,
                'Old Value': val_old,
                'New Value': val_new
            })

# --- 3. Date Format Consistency ---
def detect_date_format(date_str):
    try:
        pd.to_datetime(date_str)
        return "ISO" if '-' in date_str else "US"
    except:
        return "Invalid"

old_df['DateOfBirthFormat'] = old_df['DateOfBirth'].apply(detect_date_format)
new_df['DateOfBirthFormat'] = new_df['DateOfBirth'].apply(detect_date_format)

date_format_issues = new_df[new_df['DateOfBirthFormat'] != 'ISO']

# --- 4. Duplicate Detection ---
duplicates_old = old_df[old_df.duplicated('CustomerID', keep=False)]
duplicates_new = new_df[new_df.duplicated('CustomerID', keep=False)]

# --- Summary Metrics ---
total_records_old = len(old_df)
total_records_new = len(new_df)
missing_emails_new = missing_new['Email']
missing_phones_new = missing_new['Phone']
duplicate_count_new = len(duplicates_new)

# --- Output Results ---
print("\n=== DATA AUDIT REPORT ===\n")

print(f"Total records in old system: {total_records_old}")
print(f"Total records in new CRM: {total_records_new}")

print("\n--- MISSING VALUES ---")
print("Old System:\n", missing_old)
print("New CRM:\n", missing_new)

print("\n--- FIELD DISCREPANCIES ---")
for d in discrepancies:
    print(d)

print("\n--- DATE FORMAT ISSUES IN NEW CRM ---")
print(date_format_issues[['CustomerID', 'DateOfBirth', 'DateOfBirthFormat']])

print("\n--- DUPLICATES IN NEW CRM ---")
print(duplicates_new)

# Optional: Save discrepancies to CSV
discrepancy_df = pd.DataFrame(discrepancies)
discrepancy_df.to_csv("field_discrepancies.csv", index=False)


=== DATA AUDIT REPORT ===

Total records in old system: 4
Total records in new CRM: 4

--- MISSING VALUES ---
Old System:
 CustomerID     0
Name           0
Email          1
Phone          0
DateOfBirth    0
dtype: int64
New CRM:
 CustomerID     0
Name           0
Email          0
Phone          1
DateOfBirth    0
dtype: int64

--- FIELD DISCREPANCIES ---
{'CustomerID': '102', 'Field': 'Phone', 'Old Value': '555-0321', 'New Value': ''}
{'CustomerID': '103', 'Field': 'Email', 'Old Value': '', 'New Value': 'bob.j@oldmail.com'}
{'CustomerID': '104', 'Field': 'Name', 'Old Value': 'Alice Brown', 'New Value': ''}
{'CustomerID': '104', 'Field': 'Email', 'Old Value': 'alice.b@work.net', 'New Value': ''}
{'CustomerID': '104', 'Field': 'Phone', 'Old Value': '555-9876', 'New Value': ''}
{'CustomerID': '105', 'Field': 'Name', 'Old Value': '', 'New Value': 'Mike Davis'}
{'CustomerID': '105', 'Field': 'Email', 'Old Value': '', 'New Value': 'mike.d@newcrm.org'}
{'CustomerID': '105', 'Field': 'Phone'